#### Poetry 환경 구축 및 모듈 설치

```
poetry new "project name" -> Ex) Poetry add sutdy
poetry install
poetry env info 에 나오는 Executable 경로 가져오기기
```
1. Vscode 일 경우

Ctrl + shift + P
select interpreter
+ 가상환경 만들기
Executable의 경로 입력

In [1]:
# 필요 모듈 설치
!poetry add pandas langchain transformers torch langchain_community

The following packages are already present in the pyproject.toml and will be skipped:

  - pandas
  - langchain
  - transformers
  - torch
  - langchain_community

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


#### 모듈 호출

In [3]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


from langchain.chains import create_retrieval_chain

create_retrieval_chain?

Signature:
create_retrieval_chain(
    retriever: 'Union[BaseRetriever, Runnable[dict, RetrieverOutput]]',
    combine_docs_chain: 'Runnable[Dict[str, Any], str]',
) -> 'Runnable'
Docstring:
Create retrieval chain that retrieves documents and then passes them on.

Args:
    retriever: Retriever-like object that returns list of documents. Should
        either be a subclass of BaseRetriever or a Runnable that returns
        a list of documents. If a subclass of BaseRetriever, then it
        is expected that an `input` key be passed in - this is what
        is will be used to pass into the retriever. If this is NOT a
        subclass of BaseRetriever, then all the inputs will be passed
        into this runnable, meaning that runnable should take a dictionary
        as input.
    combine_docs_chain: Runnable that takes inputs and produces a string output.
        The inputs to this will be any original inputs to this chain, a new
        context key with the retrieved documents, and 

#### 데이터 가져오기 및 전처리

In [3]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

train.head()
test.head()

,ID,발생일시,사고인지 시간,날씨,기온,습도,공사종류,연면적,층 정보,인적사고,물적사고,공종,사고객체,작업프로세스,장소,부위,사고원인
0,TEST_000,2024-06-03 오전 09:39,정규작업 -,맑음,27℃,53%,건축 / 건축물 / 교정 및 군사시설,"1,990.32㎡","지상 1층, 지하 0층",부딪힘,전도,건축 > 철근콘크리트공사,건설기계 > 콘크리트펌프,타설작업,교정 및 군사시설 / 외부,콘크리트펌프 / 바닥,"펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측..."
1,TEST_001,2024-02-15 오전 09:00,정규작업 -,맑음,5℃,71%,건축 / 건축물 / 운수시설,"349,895㎡","지상 5층, 지하 2층","절단, 베임",없음,건축 > 수장공사,건설공구 > 공구류,절단작업,운수시설 / 내부,공구류 / 핸드그라인더,작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용
2,TEST_002,2024-02-01 오전 09:30,작업 전 -,흐림,5℃,58%,건축 / 건축물 / 공동주택,"171,198㎡","지상 27층, 지하 2층",떨어짐(2미터 미만),없음,건축 > 미장공사,기타 > 기타,이동,공동주택 / 내부,기타 / 바닥,작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐
3,TEST_003,2024-02-06 오후 05:00,정규작업 -,맑음,1℃,87%,건축 / 건축물 / 공동주택,"123,540.98㎡","지상 2층, 지하 35층",넘어짐(물체에 걸림),없음,건축 > 조적공사,건설자재 > 자재,기타,공동주택 / 102동 801호 세대 내부,자재 / 말비계(H:720mm),작업 발판 위 벽돌 잔재를 밟고 넘어짐
4,TEST_004,2024-06-03 오전 11:35,기타 -,흐림,22℃,60%,토목 / 교량 / 도로교량,-,-,떨어짐(2미터 미만),없음,토목 > 교량공사,기타 > 기타,해체작업,도로교량 / 외부,기타 / 바닥,점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생


In [4]:
train['공사종류(대분류)'] = train['공사종류'].str.split('/').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split('/').str[1]
train['공종(대분류)'] = train['공종'].str.split('>').str[0]
train['공종(중분류)'] = train['공종'].str.split('>').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split('>').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split('>').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split('/').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split('/').str[1]
test['공종(대분류)'] = test['공종'].str.split('>').str[0]
test['공종(중분류)'] = test['공종'].str.split('>').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split('>').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split('>').str[1]

In [5]:
# train.apply(
#     lambda row : print(row)
# )
train.columns

Index(['ID', '발생일시', '사고인지 시간', '날씨', '기온', '습도', '공사종류', '연면적', '층 정보',
       '인적사고', '물적사고', '공종', '사고객체', '작업프로세스', '장소', '부위', '사고원인',
       '재발방지대책 및 향후조치계획', '공사종류(대분류)', '공사종류(중분류)', '공종(대분류)', '공종(중분류)',
       '사고객체(대분류)', '사고객체(중분류)'],
      dtype='object')

In [6]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row : {
        "question" : (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중"
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서"
            f"사고객체 '{row['사고객체(대분류)']}', '중분류 : {row['사고객체(중분류)']}'와 관련된 사고가 발생했습니다."
            f"작업프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}' 입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer" : row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

combined_training_data = pd.DataFrame(list(combined_training_data))

combined_test_data = test.apply(
    lambda row : {
        "question" : (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중"
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서"
            f"사고객체 '{row['사고객체(대분류)']}', '중분류 : {row['사고객체(중분류)']}'와 관련된 사고가 발생했습니다."
            f"작업프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}' 입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
    },
    axis=1
)

combined_test_data = pd.DataFrame(list(combined_test_data))

In [7]:
# !poetry add bitsandbytes accelerate>=0.26.0

!poetry add accelerate==0.26.0 bitsandbytes

The following packages are already present in the pyproject.toml and will be skipped:

  - bitsandbytes

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.


Updating dependencies
Resolving dependencies...

No dependencies to install or update


### 모델 불러오기

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32
)

model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend